# Sign Language (ASL) recognition
An Artificial Intelligence project by Emile GATIGNON and Martin RAMPONT

> "Artificial Intelligence" - Course N° 12721 at Hanyang University with professor 백성용 / Sungyong Baik
> 
> Spring Semester 2023
